In [4]:
###############################################################################
#      Cognitive capacity and control in the evolution of intelligence 
#  
#   READ:
#   Below is the code and setting used for parameter sweep analysis 
#   This code was run on a high-performance cluster changes 
#   Change the 'reps' parameter to around 1000 for a quick workstation smoke test
#
###############################################################################

using Random
using LinearAlgebra
using Base.Threads
using Distributions
using CMAEvolutionStrategy
using DataFrames
using CSV

###############################################################################
# 1. Set parameters
###############################################################################
m_val = 3 # load or number of items 
q_val = 0.80 # cue reliability 
T_val = 2.0 # total time period length 
c_val = 0.10 # metabolic cost

println("Running with m=$(m_val), q=$(q_val), T=$(T_val), c=$(c_val)")

###############################################################################
# 2. Global settings
###############################################################################
t_prop = 0.50   # proportion of total time before the cue
Δt     = 0.01   # time step for the discrete simulation 
reps   = 2*10^6 # number of Monte Carlo replicates 
pop    = 6  # CMA-ES population size 
gen    = 40 # CMA-ES number of generations 

###############################################################################
# 3. Define the Monte Carlo simulation & expected reward calculation function
###############################################################################
function expectedR(η::Float64, κ::Float64, m::Int, q::Float64, T::Float64, t_prop::Float64, Δt::Float64, reps::Int)
    tspan = 0.0:Δt:T
    len_tspan = length(tspan)

    x̂₀ = fill(η / m, m)
    x̂₁ = clamp.(x̂₀ + log((m - 1) * q / (1 - q)) / m * vcat(m - 1, fill(-1, m - 1)), 0, η)

    δ = 1 - exp(-κ * Δt)
    σ = sqrt((1 - exp(-2 * κ * Δt)) / (2 * κ))

    tq = Int(floor(t_prop * T / Δt)) + 1  # cue-on switch index

    function Xtrad_sim(X₀::Vector{Float64}, ϵ::Matrix{Float64})
        Xt = copy(X₀)
        Xtrad = zeros(len_tspan, m)
        Xtrad[1, :] = Xt

        # pre-cue drift toward x̂₀ with noise
        for t in 2:tq-1
            a_Xt = Xt .> 0.0
            sum_a_Xt = count(a_Xt)

            raw_ΔX = δ * (x̂₀ - Xt) + σ * ϵ[t, :]
            ΔX = a_Xt .* raw_ΔX - (dot(a_Xt, raw_ΔX) / sum_a_Xt) * a_Xt

            Xt .+= ΔX
            Xtrad[t, :] = Xt
        end

        # post-cue drift toward x̂₁ with noise
        for t in tq:len_tspan
            a_Xt = Xt .> 0.0
            sum_a_Xt = count(a_Xt)

            raw_ΔX = δ * (x̂₁ - Xt) + σ * ϵ[t, :]
            ΔX = a_Xt .* raw_ΔX - (dot(a_Xt, raw_ΔX) / sum_a_Xt) * a_Xt

            Xt .+= ΔX
            Xtrad[t, :] = Xt
        end

        return Xtrad
    end

    @inline function R(x::AbstractVector{<:Real})
        # expected recall over cue types given current allocation x
        exp_neg_x = exp.(-x)
        return q * (1 - exp_neg_x[1]) +
               ((1 - q) / (m - 1)) * sum(1 .- exp_neg_x[2:end])
    end

    function R_mean(reps::Int)
        # expected recall by averaging over time and replicates
        R_sum_per_thread = zeros(Float64, nthreads())

        @threads for i in 1:reps
            X₀ = η * rand(Dirichlet(ones(m)))
            ϵ = randn(len_tspan, m)

            Xtrad_out = Xtrad_sim(X₀, ϵ)
            R_Xt = map(R, eachrow(Xtrad_out))

            R_sum_per_thread[threadid()] += sum(R_Xt)
        end

        return sum(R_sum_per_thread) / (reps * len_tspan)
    end

    return R_mean(reps)
end


###############################################################################
# 4. Define the objective and run CMA-ES
###############################################################################

function w_min(params)
    η = params[1]
    κ = params[2]
    return - (expectedR(η, κ, m_val, q_val, T_val, t_prop, Δt, reps) - c_val * (η + κ))
end

result = minimize(
    x -> w_min(x),
    [2.00, 0.50],
    1.0;
    lower = [0.001, 0.001],
    upper = [4.00, 1.00],
    noise_handling = CMAEvolutionStrategy.NoiseHandling(ασ = 1.1, callback = s -> s > 0),
    popsize = pop,
    maxiter = gen
)

final_params = xbest(result)

###############################################################################
# 5. Outputs 
###############################################################################

eta_opt   = final_params[1]
kappa_opt = final_params[2]
R_bar     = expectedR(eta_opt, kappa_opt, m_val, q_val, T_val, t_prop, Δt, reps)
info_proc_cap = log2(m_val)*R_bar


println("eta_opt=$(eta_opt), kappa_opt=$(kappa_opt), R_bar=$(R_bar), info_proc_cap=$(info_proc_cap)")



Running with m=3, q=0.8, T=2.0, c=0.1
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=64%) in dimension 2 (seed=3508345769257449819, 2025-08-28T18:27:03.789)
  iter   fevals   function value      sigma  axis ratio   time[s]
     1        8   -1.49022154e-01   1.01e+00   1.251e+00     3.309
     2       16   -1.66890164e-01   9.38e-01   1.298e+00     3.909
     3       24   -1.59045460e-01   7.27e-01   1.120e+00     4.662
     6       47   -1.60901052e-01   5.85e-01   1.238e+00     6.822
     9       71   -1.55862015e-01   1.03e+00   1.492e+00     9.054
    12       95   -1.64035483e-01   1.42e+00   2.245e+00    11.170
    14      110   -1.58067060e-01   1.14e+00   3.279e+00    12.449
eta_opt=2.7579273760467626, kappa_opt=0.05759959176170775, R_bar=0.4525698546957978, info_proc_cap=0.717306248649662


In [5]:
###############################################################################
# 6. Noise (requires previous cell run) 
###############################################################################

function noise(η::Float64, κ::Float64, m::Int, q::Float64, T::Float64,
               t_prop::Float64, Δt::Float64, reps::Int)
    tspan = 0.0:Δt:T
    L = length(tspan)

    x̂₀ = fill(η / m, m)
    x̂₁ = clamp.(x̂₀ .+ log((m - 1) * q / (1 - q)) / m .* vcat(m - 1, fill(-1, m - 1)), 0, η)

    δ = 1 - exp(-κ * Δt)
    σ = sqrt((1 - exp(-2 * κ * Δt)) / (2 * κ))
    tq = Int(floor(t_prop * T / Δt)) + 1  

    dsum_per_thread = zeros(L, nthreads())

    @threads for rep in 1:reps
        X₀ = η * rand(Dirichlet(ones(m)))
        ϵ  = randn(L, m)

        Xt_c = copy(X₀)
        Xt_n = copy(X₀)

        dloc = zeros(L)
        dloc[1] = norm(Xt_n .- Xt_c)^2

        for t in 2:(tq-1)
            a_c = Xt_c .> 0; sc = count(a_c)   
            a_n = Xt_n .> 0; sn = count(a_n)   

            raw_c = δ * (x̂₀ - Xt_c)
            raw_n = σ * ϵ[t, :]

            Δc = a_c .* raw_c .- (dot(a_c, raw_c) / sc) .* a_c
            Δn = a_n .* raw_n .- (dot(a_n, raw_n) / sn) .* a_n

            Xt_c .+= Δc
            Xt_n .+= Δn

            dloc[t] = norm(Xt_n .- Xt_c)^2
        end

        for t in tq:L
            a_c = Xt_c .> 0; sc = count(a_c)
            a_n = Xt_n .> 0; sn = count(a_n)

            raw_c = δ * (x̂₁ - Xt_c)
            raw_n = σ * ϵ[t, :]

            Δc = a_c .* raw_c .- (dot(a_c, raw_c) / sc) .* a_c
            Δn = a_n .* raw_n .- (dot(a_n, raw_n) / sn) .* a_n

            Xt_c .+= Δc
            Xt_n .+= Δn

            dloc[t] = norm(Xt_n .- Xt_c)^2
        end

        dsum_per_thread[:, threadid()] .+= dloc
    end

    dsum = vec(sum(dsum_per_thread, dims=2))
    rms_vec = sqrt.(dsum ./ reps) ./ η    
    return sum(rms_vec) / L              
end

noise_RMSD = noise(eta_opt, kappa_opt, m_val, q_val, T_val, t_prop, Δt, reps)
println("noise_RMSD=$(noise_RMSD)")


noise_RMSD=0.3849649390879382
